In [18]:
from __future__ import print_function
import numpy as np
np.random.seed(1)
import sys
import sklearn
import sklearn.ensemble
%load_ext autoreload
%autoreload 2
from anchor import utils
import lime
import lime.lime_tabular
from int_met import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
dataset_folder = 'Datasets/'
dataset = utils.load_dataset('mortality',discretize=False, balance=True, dataset_folder=dataset_folder)

In [22]:
dataset.feature_names

['age',
 'racewb',
 'sex',
 'mets_achieved',
 'resting_systolic',
 'percent_hr_achieved',
 'reason',
 'diuretic',
 'htnmed',
 'htn',
 'dm',
 'famhx',
 'smoke',
 'afib2']

In [ ]:
from copy import deepcopy, copy
def get_cat_names(cat_feats, cate_names, feat_names):
    cat_names = deepcopy(cate_names)
    for i in cat_feats:
        for j in range(len(cat_names[i])):
            cat_names[i][j] = feat_names[i] + ' = ' + cat_names[i][j]
    return cat_names

In [ ]:
cat_names = get_cat_names(list(set(dataset.categorical_features) - set(dataset.ordinal_features)), dataset.categorical_names, dataset.feature_names)

In [ ]:
cat_names

In [ ]:
flat_cat_names = [item for sublist in cat_names.values() for item in sublist]

In [ ]:
flat_cat_names

In [15]:
dataset.feature_names

['age',
 'racewb',
 'sex',
 'mets_achieved',
 'resting_systolic',
 'percent_hr_achieved',
 'reason',
 'diuretic',
 'htnmed',
 'htn',
 'dm',
 'famhx',
 'smoke',
 'afib2']

In [28]:
dataset.labels_train

array([1, 1, 1, ..., 0, 1, 0], dtype=int64)

In [23]:
dataset.data.shape

(8360, 14)

In [134]:
explainer = lime.lime_tabular.LimeTabularExplainer(dataset.train, feature_names=dataset.feature_names, class_names=dataset.class_target,discretize_continuous=False,)

In [139]:
c = sklearn.ensemble.RandomForestClassifier(n_estimators=70, n_jobs=5, random_state=1)
c.fit(dataset.train, dataset.labels_train)
predict_fn = lambda x: c.predict(x)
print('Train', sklearn.metrics.accuracy_score(dataset.labels_train, predict_fn(dataset.train)))
print('Test', sklearn.metrics.accuracy_score(dataset.labels_test, predict_fn(dataset.test)))

Train 1.0
Test 0.8504784688995215


In [151]:
i = np.random.randint(0, dataset.test.shape[0])
exp = explainer.explain_instance(dataset.test[i], c.predict_proba, num_features=5, top_labels=1)

In [226]:
exp_fn = lambda i: explainer.explain_instance(dataset.test[i], c.predict_proba, num_features=5, top_labels=1)
def exp_fn_blk(xtest, exp_fn):
    exp1 = []
    for i in range(len(xtest)):
        #exp = explainer.explain_instance(dataset.test[i], c.predict_proba, num_features=5, top_labels=1)
        exp = exp_fn(i)
        exp1.append(exp.as_map()[exp.available_labels()[0]])
    return np.array(exp1)
exp_fn_wrap = lambda x: np.array(exp_fn_blk(x, exp_fn))

In [219]:
expp = exp_fn_wrap(dataset.test[:10])

In [251]:
exp1 = exp_fn_blk(dataset.test[:100], exp_fn)
exp2 = exp_fn_blk(dataset.test[:100], exp_fn)

In [228]:
i = calc_identity(exp1,exp2)
print(i)

(100.0, 0, 10)


In [229]:
s = calc_separability(dataset.test[:100], exp_fn_wrap)
print(s)

(0, 9604, 0.0)


In [240]:
len(dataset.feature_names)

14

In [244]:
def enc_exp(exp, feature_num):
    enc_exp = np.zeros((len(exp),feature_num))
    for i in range(len(exp)):
        for j in range(len(exp[i])):
            enc_exp[i][int(exp[i,j,0])] = exp[i,j,1]
    return enc_exp

In [252]:
enc1 = enc_exp(exp1, len(dataset.feature_names))
enc2 = enc_exp(exp2, len(dataset.feature_names))

In [254]:
enc1.shape

(100, 14)

In [255]:
sb = calc_stability(enc1, dataset.labels_test[:100])
print(sb)

C:\Users\youss\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\sklearn\cluster\k_means_.py:896: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)


(21, 100)


In [205]:
np.array(exp).shape

(10, 5, 2)

In [204]:
np.array(exp1).shape

(10, 5, 2)

In [187]:
exp.as_map()[1]

[(0, 0.13793875574327166),
 (3, -0.09261587474365167),
 (5, -0.04429587682338041),
 (2, 0.02894435619889872),
 (12, 0.027213078075854536)]

In [190]:
np.array_equal(exp1[8],exp.as_map()[1])

False

In [162]:
exp.as_map()[exp.available_labels()[0]][0]

(0, 0.13858095215414457)

In [179]:
exp1.as_map()

array(dict_values([[(0, -0.12926717218402034), (3, 0.0892935833836895), (5, 0.04466194710585964), (2, -0.02731541614619599), (12, -0.023873271098755932), (4, -0.014467186070004516), (9, -0.013834473329186137), (10, -0.010319174696956623), (1, 0.005975404429819767), (8, -0.005902964182266721)]]),
      dtype=object)

In [156]:
exp==exp1

False

In [153]:
exp.as_list(label = exp.available_labels()[0])

[('age', 0.13858095215414457),
 ('mets_achieved', -0.09012903252072652),
 ('percent_hr_achieved', -0.03697454376266478),
 ('sex', 0.031058386188899317),
 ('smoke', 0.02766903580186015)]

In [163]:
a = np.array(exp1.as_map()[exp1.available_labels()[0]])

In [171]:
b = np.array([(0, -0.12926717218402034),
 (3, 0.0892935833836895),
 (5, 0.04466194710585964),
 (2, -0.02731541614619599),
 (12, -0.023873271098755932),
 (4, -0.014467186070004516),
 (9, -0.013834473329186137),
 (10, -0.010319174696956623),
 (1, 0.005975404429819767),
 (8, -0.005902964182266721)])

In [173]:
a.shape

(10, 2)

In [172]:
np.array_equal(a,b)

True

In [ ]:
explainer = anchor_tabular.AnchorTabularExplainer(dataset.class_names, dataset.feature_names, dataset.data, dataset.categorical_names)
explainer.fit(dataset.train, dataset.labels_train, dataset.validation, dataset.labels_validation)

In [25]:
c = sklearn.ensemble.RandomForestClassifier(n_estimators=100, n_jobs=5, random_state= 46)
c.fit(explainer.encoder.transform(dataset.train), dataset.labels_train)
predict_fn = lambda x: c.predict(explainer.encoder.transform(x))
print('Train', sklearn.metrics.accuracy_score(dataset.labels_train, predict_fn(dataset.train)))
print('Test', sklearn.metrics.accuracy_score(dataset.labels_test, predict_fn(dataset.test)))
print('Train', sklearn.metrics.roc_auc_score(dataset.labels_train, predict_fn(dataset.train)))
print('Test', sklearn.metrics.roc_auc_score(dataset.labels_test, predict_fn(dataset.test)))

AttributeError: 'LimeTabularExplainer' object has no attribute 'encoder'

In [ ]:
exp_fn_bulk = lambda x: np.array([np.array(explainer.explain_instance(x[idx], c.predict, threshold=0.95).names()) for idx in range(len(x))])

In [ ]:
exp_lss = exp_fn_bulk(dataset.test[:100])
exp_lss1 = exp_fn_bulk(dataset.test[:100])

In [ ]:
exp1 = [np.array(exp_lss[i].names()) for i in range(len(exp_lss))]
exp2 = [np.array(exp_lss1[i].names()) for i in range(len(exp_lss1))]

In [ ]:
i = calc_identity(exp1,exp2)
print(i)

In [ ]:
s = calc_separability(dataset.test[:100], exp_fn_bulk)
print (s)

In [ ]:
def encode_exp(exp, flat_cat_names):
    enc_exp = np.zeros((1,len(flat_cat_names)))
    for i in range(len(flat_cat_names)):
        if flat_cat_names[i] in exp:
            enc_exp[0,i] = 1
    return enc_exp

In [ ]:
enc_exps = np.array([encode_exp(exp1[i], flat_cat_names) for i in range(len(exp1))]).squeeze()

In [ ]:
sb = calc_stability(enc_exps, dataset.labels_test[:100])
print(sb)

In [ ]:
idx = 5
np.random.seed(1)
print('Prediction: ', explainer.class_names[predict_fn(dataset.test[idx].reshape(1, -1))[0]])
exp = explainer.explain_instance(dataset.test[idx], c.predict, threshold=0.95)

In [ ]:
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print('Coverage: %.2f' % exp.coverage())

In [ ]:
# Get test examples where the anchora pplies
fit_anchor = np.where(np.all(dataset.test[:, exp.features()] == dataset.test[idx][exp.features()], axis=1))[0]
print('Anchor test coverage: %.2f' % (fit_anchor.shape[0] / float(dataset.test.shape[0])))
print('Anchor test precision: %.2f' % (np.mean(predict_fn(dataset.test[fit_anchor]) == predict_fn(dataset.test[idx].reshape(1, -1)))))

In [ ]:
print('Partial anchor: %s' % (' AND '.join(exp.names(1))))
print('Partial precision: %.2f' % exp.precision(1))
print('Partial coverage: %.2f' % exp.coverage(1))

In [ ]:
fit_partial = np.where(np.all(dataset.test[:, exp.features(1)] == dataset.test[idx][exp.features(1)], axis=1))[0]
print('Partial anchor test precision: %.2f' % (np.mean(predict_fn(dataset.test[fit_partial]) == predict_fn(dataset.test[idx].reshape(1, -1)))))
print('Partial anchor test coverage: %.2f' % (fit_partial.shape[0] / float(dataset.test.shape[0])))

In [ ]:
exp.show_in_notebook()